In [1]:
import pandas as pd

data = pd.read_parquet('../cache/btc.parquet')
data.head()

,Timestamp,Open,High,Low,Close,Volume
0,1.325412e+09,4.58,4.58,4.58,4.58,0.0
1,1.325412e+09,4.58,4.58,4.58,4.58,0.0
2,1.325412e+09,4.58,4.58,4.58,4.58,0.0
3,1.325412e+09,4.58,4.58,4.58,4.58,0.0
4,1.325412e+09,4.58,4.58,4.58,4.58,0.0


In [2]:
# show all columns with nan values
data.isna().sum()

Timestamp    1
Open         0
High         0
Low          0
Close        0
Volume       0
dtype: int64

In [3]:
# drop columns that are not needed
data = data.drop(columns=['Volume', 'High', 'Low', 'Close'])

In [4]:
# set index to Datetime and sample every 15min interval
data['Datetime'] = pd.to_datetime(data['Timestamp'], unit='s')
data.set_index('Datetime', inplace=True)

# have a small buffer of 30 min at the end for change % calculation
date_range = pd.date_range(start='2019-01-01 00:00:00', end='2024-01-02 00:00:00', freq='15min')

data = data.reindex(date_range)
data.head()

,Timestamp,Open
2019-01-01 00:00:00,1.546301e+09,3750.62
2019-01-01 00:15:00,1.546302e+09,3749.38
2019-01-01 00:30:00,1.546303e+09,3742.13
2019-01-01 00:45:00,1.546304e+09,3652.70
2019-01-01 01:00:00,1.546304e+09,3685.65


In [5]:
# correct all timestamps with the help of Datetime and convert timestamp to int64
data['Timestamp'] = data.index.astype('int64') // 10**9

In [6]:
# fill NaN values with interpolation
data = data.interpolate()

In [7]:
# show all columns with nan values
data.isna().sum()

Timestamp    0
Open         0
dtype: int64

In [8]:
# Calculate the percentage change for 15 min intervals
data['Open_shifted'] = data['Open'].shift(-1)
data['pct_change_15min'] = ((data['Open_shifted'] - data['Open']) / data['Open']) * 100
data.head()

,Timestamp,Open,Open_shifted,pct_change_15min
2019-01-01 00:00:00,1546300800,3750.62,3749.38,-0.033061
2019-01-01 00:15:00,1546301700,3749.38,3742.13,-0.193365
2019-01-01 00:30:00,1546302600,3742.13,3652.70,-2.389815
2019-01-01 00:45:00,1546303500,3652.70,3685.65,0.902072
2019-01-01 01:00:00,1546304400,3685.65,3681.25,-0.119382


In [9]:
data['Open_shifted'] = data['Open'].shift(-2)

# Calculate the percentage change for 30 min intervals
data['pct_change_30min'] = ((data['Open_shifted'] - data['Open']) / data['Open']) * 100
data.head()

,Timestamp,Open,Open_shifted,pct_change_15min,pct_change_30min
2019-01-01 00:00:00,1546300800,3750.62,3742.13,-0.033061,-0.226363
2019-01-01 00:15:00,1546301700,3749.38,3652.70,-0.193365,-2.578560
2019-01-01 00:30:00,1546302600,3742.13,3685.65,-2.389815,-1.509301
2019-01-01 00:45:00,1546303500,3652.70,3681.25,0.902072,0.781614
2019-01-01 01:00:00,1546304400,3685.65,3674.89,-0.119382,-0.291943


In [10]:
# calculate the percentage change for 24 hours intervals
data['Open_shifted'] = data['Open'].shift(-96)
data['pct_change_24h'] = ((data['Open_shifted'] - data['Open']) / data['Open']) * 100
data.head()

,Timestamp,Open,Open_shifted,pct_change_15min,pct_change_30min,pct_change_24h
2019-01-01 00:00:00,1546300800,3750.62,3659.35,-0.033061,-0.226363,-2.433464
2019-01-01 00:15:00,1546301700,3749.38,3678.45,-0.193365,-2.578560,-1.891779
2019-01-01 00:30:00,1546302600,3742.13,3675.67,-2.389815,-1.509301,-1.775994
2019-01-01 00:45:00,1546303500,3652.70,3685.42,0.902072,0.781614,0.895776
2019-01-01 01:00:00,1546304400,3685.65,3698.79,-0.119382,-0.291943,0.356518


In [11]:
data = data.drop(columns=['Open', 'Open_shifted'])

# remove the last 2 rows as they have NaN values
date_range = pd.date_range(start='2019-01-01 00:00:00', end='2023-12-31 23:45:00', freq='15min')
data = data.reindex(date_range)
data = data.asfreq('15min')

data.tail()

,Timestamp,pct_change_15min,pct_change_30min,pct_change_24h
2023-12-31 22:45:00,1704062700,0.195193,0.103476,0.500917
2023-12-31 23:00:00,1704063600,-0.091539,0.044596,0.539843
2023-12-31 23:15:00,1704064500,0.136259,0.267819,1.296810
2023-12-31 23:30:00,1704065400,0.131381,0.065691,1.029936
2023-12-31 23:45:00,1704066300,-0.065604,-0.131209,1.126992


In [12]:
from datetime import datetime, timezone

date_time = datetime.fromtimestamp(data['Timestamp'].iloc[0], tz=timezone.utc)
print("Start date: ", date_time)
date_time = datetime.fromtimestamp(data['Timestamp'].iloc[1], tz=timezone.utc)
print("Second date: ", date_time)
date_time = datetime.fromtimestamp(data['Timestamp'].max(), tz=timezone.utc)
print("End date: ", date_time)

Start date:  2019-01-01 00:00:00+00:00
Second date:  2019-01-01 00:15:00+00:00
End date:  2023-12-31 23:45:00+00:00


In [13]:
data.to_parquet("../cache/btc_cleaned.parquet")